## python script to improve poor quality of images

In [1]:
!pip install opencv-python
!pip install numpy

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [ ]:
input_dir = 'unified_data/normal'
output_dir = 'unified_data/normal'

# Check if folders exist and list some files
print(" Input dir exists:", os.path.exists(input_dir))
print(" Files found:", len(os.listdir(input_dir)))
print(" Sample files:", os.listdir(input_dir)[:5])


In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm

In [ ]:
def enhance_image(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    denoised = cv2.fastNlMeansDenoisingColored(enhanced, None, 10, 10, 7, 21)
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    sharpened = cv2.filter2D(denoised, -1, kernel)
    return sharpened

# Make sure output dir exists
os.makedirs(output_dir, exist_ok=True)

# Check processing with progress bar and error catching
for i, fname in enumerate(tqdm(os.listdir(input_dir))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    try:
        img_path = os.path.join(input_dir, fname)
        img = cv2.imread(img_path)

        if img is None:
            print(f" Skipping unreadable file: {fname}")
            continue

        out = enhance_image(img)
        cv2.imwrite(os.path.join(output_dir, fname), out)

        if i % 50 == 0:
            print(f" Processed: {fname}")

    except Exception as e:
        print(f" Error processing {fname}: {e}")

print(" All images processed.")